<a href="https://colab.research.google.com/github/MathMachado/DSWP/blob/master/Notebooks/NB10_04__3DP_2_Missing_Value_Handling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><h1><b><i>3DP_2 - MISSING VALUES HANDLING</i></b></h1></center>


___
# **REFERÊNCIAS**
* [Working with missing data](https://pandas.pydata.org/pandas-docs/stable/user_guide/missing_data.html)
* [Handling Missing Data for a Beginner](https://towardsdatascience.com/handling-missing-data-for-a-beginner-6d6f5ea53436)

___
# **3DP_MISSING VALUES HANDLING**

> Lidar com Missing Values é um dos piores pesadelos de um Cientista de dados. Especialmente, se o número de MV for grande o suficiente (geralmente acima de 5%). Nesse caso, os valores não podem ser descartados e um Cientista de Dados inteligente deve "imputar" os valores ausentes.

* Nesta sessão, vamos identificar, analisar e tratar Missing Values (MV).
* Como MV são gerados?
    * Usuário se esqueceu de preencher ou preencheu errado o campo;
    * Os dados foram perdidos durante a transferência manual de um banco de dados legado;
    * Erro de programação;
    * Os usuários optaram por não preencher um campo vinculado a suas crenças sobre como os resultados seriam usados ou interpretados.
* As funções df.isnull() e df.isna() são apropriadas para nos indicar quantas observações são MV no dataframe.

* Na prática:
    * Variáveis Contínuas/Numéricas - Podemos substituir os NaN por Média/Mediana/Moda;
	* Variáveis Categóricas - Uma alternativa é atribuir uma categoria inexistente como, por exemplo "MV" para indicar o NaN.


___
# **MACHINE LEARNING COM PYTHON (Scikit-Learn)**

![Scikit-Learn](https://github.com/MathMachado/Materials/blob/master/scikit-learn-1.png?raw=true)

## Carregar as biliotecas

In [54]:
import pandas as pd
from pandas import Series, DataFrame

import numpy as np
from sklearn import preprocessing
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
matplotlib.style.use('ggplot')

# remove warnings to keep notebook clean
import warnings
warnings.filterwarnings('ignore')

## Dataframes
* O dataframe abaixo foi gerado aleatoriamente para entendermos como lidar com os NaN's.

In [55]:
df= pd.DataFrame({
    'idade': [32,38,np.nan,37,np.nan,36,38,32,0,np.nan],
    'salario': ['High', 'High', 'High', 'Low', 'Low', 'High', np.nan, 'Medium', 'Medium', 'High'],
    'pais': ['Spain', 'France', 'France', np.nan, 'Germany', 'France', 'Spain', 'France', np.nan, 'Spain']})

df

,idade,salario,pais
0,32.0,High,Spain
1,38.0,High,France
2,NaN,High,France
3,37.0,Low,NaN
4,NaN,Low,Germany
5,36.0,High,France
6,38.0,NaN,Spain
7,32.0,Medium,France
8,0.0,Medium,NaN
9,NaN,High,Spain


## Identificar os NaN's

A função df.isna() será usada para identificarmos os NaN's nos dataframes. Por exemplo:

In [56]:
df.isna()

,idade,salario,pais
0,False,False,False
1,False,False,False
2,True,False,False
3,False,False,True
4,True,False,False
5,False,False,False
6,False,True,False
7,False,False,False
8,False,False,True
9,True,False,False


Qual a interpretação deste output?

Para um dataframe muito grande, vamos usar a expressão abaixo:

In [57]:
df.isna().sum()

,0
idade,3
salario,1
pais,2


In [58]:
df.isna().sum()[2]

2

Mais prático não é? No entanto, vamos utilizar a função abaixo, que nos ajudará mais com os NaN's:

In [59]:
def mostra_missing_value(df):
    total = df.isnull().sum().sort_values(ascending = False)
    percent = 100*round((df.isnull().sum()/df.isnull().count()).sort_values(ascending = False), 2)
    missing_data = pd.concat([total, percent], axis = 1, keys=['Total', 'Percentual'])
    print(missing_data.head(10))

In [60]:
mostra_missing_value(df)

         Total  Percentual
idade        3        30.0
pais         2        20.0
salario      1        10.0


## A função df.dropna()
* Esta função deleta as instâncias (linhas do dataframes) onde há pelo menos 1 NaN.

In [61]:
df

,idade,salario,pais
0,32.0,High,Spain
1,38.0,High,France
2,NaN,High,France
3,37.0,Low,NaN
4,NaN,Low,Germany
5,36.0,High,France
6,38.0,NaN,Spain
7,32.0,Medium,France
8,0.0,Medium,NaN
9,NaN,High,Spain


In [62]:
df2 = df.dropna()
df2

,idade,salario,pais
0,32.0,High,Spain
1,38.0,High,France
5,36.0,High,France
7,32.0,Medium,France


Como podemos ver, somente as instâncias 0, 1, 5 e 7 tem atributos não NaN's.

Uma forma menos severa seria:

In [63]:
df3 = df.dropna(axis = 0, subset = ['pais'])
df3

,idade,salario,pais
0,32.0,High,Spain
1,38.0,High,France
2,NaN,High,France
4,NaN,Low,Germany
5,36.0,High,France
6,38.0,NaN,Spain
7,32.0,Medium,France
9,NaN,High,Spain


* Saberias explicar o que o comando acima fez?

## Tratar os NaN's de Variáveis Numéricas
* Neste exemplo, vou substituir os NaN's da variável 'idade' pela mediana. No entanto, responda a seguinte perfunta:
    * Faz sendido idade= 0?

Acho que a resposta é não. Então, neste caso, 0 é um NaN. Vamos substituído pela mediana da variável:

In [64]:
df

,idade,salario,pais
0,32.0,High,Spain
1,38.0,High,France
2,NaN,High,France
3,37.0,Low,NaN
4,NaN,Low,Germany
5,36.0,High,France
6,38.0,NaN,Spain
7,32.0,Medium,France
8,0.0,Medium,NaN
9,NaN,High,Spain


In [65]:
df['idade2'] = df['idade'].replace({0: df['idade'].median()})
df

,idade,salario,pais,idade2
0,32.0,High,Spain,32.0
1,38.0,High,France,38.0
2,NaN,High,France,NaN
3,37.0,Low,NaN,37.0
4,NaN,Low,Germany,NaN
5,36.0,High,France,36.0
6,38.0,NaN,Spain,38.0
7,32.0,Medium,France,32.0
8,0.0,Medium,NaN,36.0
9,NaN,High,Spain,NaN


Como podemos verificar acima na variável 'idade2', o valor 0 foi substituído pela mediana da variável 'idade'.

Vamos verificar a média da variável antes da operação:

In [66]:
df['idade2'].mean()

35.57142857142857

In [67]:
df['idade3'] = df['idade2']
df

,idade,salario,pais,idade2,idade3
0,32.0,High,Spain,32.0,32.0
1,38.0,High,France,38.0,38.0
2,NaN,High,France,NaN,NaN
3,37.0,Low,NaN,37.0,37.0
4,NaN,Low,Germany,NaN,NaN
5,36.0,High,France,36.0,36.0
6,38.0,NaN,Spain,38.0,38.0
7,32.0,Medium,France,32.0,32.0
8,0.0,Medium,NaN,36.0,36.0
9,NaN,High,Spain,NaN,NaN


Aplicamos a operação:

In [68]:
df['idade3'].fillna(df['idade3'].median(), inplace = True)
df

,idade,salario,pais,idade2,idade3
0,32.0,High,Spain,32.0,32.0
1,38.0,High,France,38.0,38.0
2,NaN,High,France,NaN,36.0
3,37.0,Low,NaN,37.0,37.0
4,NaN,Low,Germany,NaN,36.0
5,36.0,High,France,36.0,36.0
6,38.0,NaN,Spain,38.0,38.0
7,32.0,Medium,France,32.0,32.0
8,0.0,Medium,NaN,36.0,36.0
9,NaN,High,Spain,NaN,36.0


Podemos observar que os valores NaN's do atributo 'idade3' foi substituído pelo valor 36.

E agora, a média após a operação:

In [69]:
df['idade3'].mean()

35.7

* Qual a conclusão?
    * Houve muito impacto na distribuição da variável 'idade'?

## Tratar NaN's de Variáveis Categóricas
* Observe a variável 'pais'. Temos alguns NaN's. As alternativas que temos são:
    * substituir os NaN's desta variável pela moda (valor mais frequente) da distribuição.
    * substiruir os NaN's por 'Undefined'.

Qual o valor (no caso, País) mais frequente ?

In [70]:
df.pais.value_counts()

,count
pais,
France,4
Spain,3
Germany,1


Ok, a instância 'France' é o mais frequente. Então vamos substituir os NaN's por 'France'. De forma automática, temos:

In [71]:
s_pais_mais_frequente = df['pais'].mode()[0]
s_pais_mais_frequente

'France'

In [72]:
df["pais2"] = df["pais"]
df["pais2"] = df["pais2"].fillna(s_pais_mais_frequente)
df

,idade,salario,pais,idade2,idade3,pais2
0,32.0,High,Spain,32.0,32.0,Spain
1,38.0,High,France,38.0,38.0,France
2,NaN,High,France,NaN,36.0,France
3,37.0,Low,NaN,37.0,37.0,France
4,NaN,Low,Germany,NaN,36.0,Germany
5,36.0,High,France,36.0,36.0,France
6,38.0,NaN,Spain,38.0,38.0,Spain
7,32.0,Medium,France,32.0,32.0,France
8,0.0,Medium,NaN,36.0,36.0,France
9,NaN,High,Spain,NaN,36.0,Spain


In [ ]:
df["pais3"] = df["pais"].fillna('pais_mv')

In [ ]:
df

# **EXERCÍCIOS**
A library Faker foi utilizada para gerar um dataframe sintético com 5.000 linhas e 25 colunas para que vocês possam praticar a identificação e tratamento dos Missing Values.


In [ ]:
%pip install faker
from faker import Faker

In [ ]:
# Initializar a library Faker e random seed
fake = Faker()
Faker.seed(42)

# Gerar um dataframe com 5000 linhas e 25 colunas
n_rows = 5000
data = {
    'ID': range(1, n_rows + 1),
    'Name': [fake.name() for _ in range(n_rows)],
    'Age': [fake.random_int(min=18, max=70) for _ in range(n_rows)],
    'Gender': [fake.random_element(elements=('Male', 'Female')) for _ in range(n_rows)],
    'Occupation': [fake.job() for _ in range(n_rows)],
    'Income': [fake.random_int(min=20000, max=120000) for _ in range(n_rows)],
    'Marital_Status': [fake.random_element(elements=('Single', 'Married', 'Divorced', 'Widowed')) for _ in range(n_rows)],
    'Credit_Score': [fake.random_element(elements=[700, 650, 600, 550, None]) for _ in range(n_rows)],
    'Loan_Amount': [fake.random_int(min=5000, max=50000) for _ in range(n_rows)],
    'Property_Value': [fake.random_int(min=50000, max=500000) for _ in range(n_rows)],
    'State': [fake.state() for _ in range(n_rows)],
    'Country': [fake.country() for _ in range(n_rows)],
    'ZIP_Code': [fake.zipcode() for _ in range(n_rows)],
    'Phone_Number': [fake.phone_number() for _ in range(n_rows)],
    'Email': [fake.email() for _ in range(n_rows)],
    'Registration_Date': [fake.date_this_decade() for _ in range(n_rows)],
    'Last_Purchase_Date': [fake.date_this_year() for _ in range(n_rows)],
    'Number_of_Purchases': [fake.random_int(min=0, max=100) for _ in range(n_rows)],
    'Loyalty_Points': [fake.random_int(min=0, max=5000) for _ in range(n_rows)],
    'Subscription_Status': [fake.random_element(elements=('Active', 'Inactive', 'Pending', 'Cancelled')) for _ in range(n_rows)],
    'Feedback_Score': [fake.random_element(elements=[1, 2, 3, 4, 5, None]) for _ in range(n_rows)],
    'Preferred_Contact_Method': [fake.random_element(elements=('Email', 'Phone', 'Mail', 'SMS')) for _ in range(n_rows)],
    'Promotions_Accepted': [fake.random_int(min=0, max=50) for _ in range(n_rows)],
    'Referrals': [fake.random_int(min=0, max=10) for _ in range(n_rows)],
    'Account_Balance': [fake.random_int(min=0, max=100000) for _ in range(n_rows)],
    'Annual_Expenses': [fake.random_int(min=10000, max=80000) for _ in range(n_rows)],
}

# Converter para dataframe
df_large = pd.DataFrame(data)

# Introduzindo Missing Values em várias colunas
columns_with_nan = ['Age', 'Occupation', 'Income', 'Credit_Score', 'Loan_Amount', 'Feedback_Score', 'Last_Purchase_Date', 'ZIP_Code', 'Loyalty_Points', 'Country']
for column in columns_with_nan:
    mask = np.random.rand(n_rows) < 0.1  # 10% missing
    df_large.loc[mask, column] = np.nan

df_large.head()

1. Contagem de Missing Values
   - Use funções apropriadas para para contar a quantidade total de Missing Values em cada coluna do `df_large`. Identifique quais colunas possuem mais MVs e quantos valores estão ausentes em cada uma.

2. Percentual de Missing Values por Coluna
   - Calcule o percentual de Missing Values em cada coluna e identifique quais colunas têm mais de 10% de valores ausentes. Liste essas colunas e o percentual de MVs em cada uma.

3. Remoção de Colunas com Alta Taxa de Missing Values
   - Remova as colunas que possuem mais de 10% de Missing Values do DataFrame e verifique como a remoção impacta o tamanho e a integridade dos dados.

4. Visualização dos Missing Values
   - Usando a biblioteca `missingno` ou `seaborn`, crie uma visualização gráfica (como um heatmap) que mostre onde os Missing Values estão distribuídos no `df_large`. Interprete o gráfico para identificar padrões na ausência de dados.

5. Imputação de Variáveis Numéricas com Média
   - Para colunas numéricas (`Age`, `Income`, `Loan_Amount`), substitua os valores ausentes pela média da coluna usando `fillna()`. Analise o impacto da imputação na média e mediana da coluna.

6. Imputação de Variáveis Numéricas com Mediana
   - Agora, substitua os valores ausentes nas colunas numéricas `Income` e `Loan_Amount` pela mediana dessas colunas. Compare os resultados com a imputação feita com a média e discuta qual estratégia foi mais apropriada.

7. Imputação de Variáveis Categóricas
   - Para variáveis categóricas com valores ausentes (`Occupation`, `Feedback_Score`, `Credit_Score`), substitua os MVs por uma nova categoria chamada `"Unknown"` usando `fillna("Unknown")`. Analise como a distribuição das categorias foi afetada.

8. Comparação entre Métodos de Imputação para Variáveis Numéricas
   - Compare os métodos de imputação com média, mediana e interpolação linear para uma coluna numérica com MVs, como `Age`. Visualize a distribuição da variável antes e depois de cada método de imputação e discuta qual método parece mais apropriado para esses dados.

9. Criação de Função para Imputação
   - Crie uma função que receba uma coluna e um método de imputação ("média", "mediana", "interpolação", ou "nova categoria") e substitua os MVs conforme o método selecionado. Aplique a função em várias colunas do DataFrame.

10. Análise de Correlação após a Imputação
   - Depois de realizar a imputação de MVs nas colunas numéricas, use `df.corr()` para verificar as correlações entre variáveis. Compare as correlações antes e depois da imputação e analise se houve mudanças significativas na relação entre as variáveis.

Esses exercícios vão ajudar os alunos a entender as várias abordagens para lidar com Missing Values, com ênfase na escolha dos métodos apropriados de imputação para cada tipo de dado.

## Exercício 11 - Titanic
> Trate os NaN's do dataframe Titanic